# Clustering ICD9 Codes

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
np.set_printoptions(threshold=np.nan)
from sklearn import random_projection
from kmodes.kmodes import KModes

In [ ]:
df = pd.read_csv('DIAGNOSES_ICD.csv',usecols=['HADM_ID','ICD9_CODE'])
df['ICD9_CODE'] = df['ICD9_CODE'].astype(str)
#df = df.head(1000)

In [ ]:
le = LabelEncoder()
le.fit(df['ICD9_CODE'])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
#print(le_name_mapping)

labelencoder = LabelEncoder()
df['ICD9_CODE'] = labelencoder.fit_transform(df['ICD9_CODE'])
#print(df)

In [ ]:
patient_list = []
for index, row in df.iterrows():
    if(row['ICD9_CODE'] == 824):
        patient_list.append(row['HADM_ID'])
#print(patient_list)

In [ ]:
Patient_lib = {}
for index, row in df.iterrows():
    code = row['ICD9_CODE']
    patient = row['HADM_ID']
    if(patient in patient_list):
        Patient_lib.setdefault(patient, []).append(code)
#print(Patient_lib)

In [ ]:
hot_df = pd.read_csv('hot_df.csv')
'''
hot_df = pd.DataFrame()
for patient, code_list in Patient_lib.items():
    patient_codes = {}
    for code in code_list:
        patient_codes[code] = 1
    hot_df = hot_df.append(patient_codes,ignore_index=True)
    print(patient)
hot_df.to_csv('hot_df.csv')
'''
print(hot_df.shape)

In [ ]:
hot_df.fillna(0)
hot_df = hot_df.drop(columns=['patients'])
code_list = list(hot_df)
for code in code_list:  
    hot_df.at['Total',code] = hot_df[code].sum()
    #print(code)
#print(hot_df)

In [ ]:
#print(hot_df)
for code in code_list:
    if((hot_df.at['Total',code]) < 100):
        hot_df = hot_df.drop(columns=[code])
        #print(code)
print(hot_df.shape)

In [ ]:
hot_df.to_csv('min_hot_df.csv')

In [2]:
min_hot_df = pd.read_csv('min_hot_df.csv')

In [3]:
min_hot_df.fillna(0)
min_hot_df = min_hot_df.drop(columns=['patients'])
code_list = list(min_hot_df)
for code in code_list:
    if((min_hot_df.at[9057,code]) < 200):
        min_hot_df = min_hot_df.drop(columns=[code])
        #print(code)
print(min_hot_df.shape)

(9058, 105)


In [4]:
min_hot_df.to_csv('min_hot_df.csv')

In [6]:
min_hot_df = pd.read_csv('min_hot_df.csv')
#print(min_hot_df)
min_hot_df = min_hot_df.drop(columns=['patients','824'])

In [7]:
#print(hot_df.shape)
np_min_hot_df = min_hot_df.values
np_min_hot_df = np.nan_to_num(np_min_hot_df)
#transformer = random_projection.SparseRandomProjection(eps=.4)
#new_hot_df = transformer.fit_transform(np_hot_df)
print(np_min_hot_df.shape)


(9058, 104)


In [10]:
km = KModes(n_clusters=2, init='Cao',n_init=5,verbose=1)
clusters=km.fit_predict(np_min_hot_df)

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 484, cost: 62764.0
Run 1, iteration: 2/100, moves: 34, cost: 62764.0


In [11]:
print(km.cluster_centroids_)

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


icd9code:
1st cluster - 1962 -> 401.9 - unspecified essential hypertension

2nd cluster - 2109 -> 428.1 - left heart failure
              88   -> 038.9 - unspecified septicemia
              2957 -> 584.9 - acute kidney failure, unspecified
              5873 -> 995.92 - severe sepsis -> caused by sig deop in bp
              2098 -> 427.31 - atrail fibrillation -> irregular heartbeat
              4451 -> 785.52 - septic shock -> symptom low bp